In [1]:
import pyro
from pyro import poutine
import pyro.distributions as dist
import torch

outer_plate=pyro.plate(5)

def model(xinds):
    scale_tensor=torch.bincount(xinds, minlength=5).view(-1,1,1,)+1e-10
    with poutine.scale(scale=scale_tensor):
        with outer_plate:
            with pyro.plate(6):
                with pyro.plate(1000):
                    pyro.sample('param_with_cost_sample',dist.Laplace(torch.zeros(5,6,1000),torch.ones(5,6,1000)))


def guide(xinds):
    scale_tensor=torch.bincount(xinds, minlength=5).view(-1,1,1,)+1e-10
    pwc=pyro.param('param_with_cost',torch.ones(5,6,1000))
    with poutine.scale(scale=scale_tensor):
        with outer_plate:
            with pyro.plate(6):
                with pyro.plate(1000):
                    pyro.sample('param_with_cost_sample',dist.Delta(pwc))

pyro.clear_param_store()
optim = pyro.optim.Adam({"lr": 0.1})
elbo = pyro.infer.Trace_ELBO(max_plate_nesting=3)
svi = pyro.infer.SVI(model, guide, optim, loss=elbo)
for i in range(10):
    xinds=torch.randint(0, 4, (300,))
    loss = svi.step(xinds)
    print(loss)


/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/utils/miniconda3/envs/pyro/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3047664.5
2867664.25
2688100.25
2508629.75
2329102.25
2149821.75
1970535.125
1791202.875
1612103.875
1432832.75


In [23]:
reshape=[]
torch.rand(5,6)[indexing_none_list(5)].shape

torch.Size([1, 1, 1, 1, 1, 5, 6])